In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructField, StructType, StringType, BooleanType, IntegerType
from src.configs import *

In [3]:
spark = SparkSession.builder \
    .appName("WikiStreamProcessor") \
    .master("local[*]") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/17 19:38:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
# Function to read data from socket using PySpark
def read_from_socket(spark):
    # Define schema for incoming JSON data
    schema = StructType([
        StructField("id", StringType(), True),
        StructField("title", StringType(), True),
        StructField("timestamp", StringType(), True),
        StructField("user", StringType(), True),
        StructField("bot", BooleanType(), True),
        StructField("minor", BooleanType(), True),
        StructField("change", IntegerType(), True),
        StructField("comment", StringType(), True),
    ])

    # Read from socket
    print(f"Connecting to socket at {SOCKET_HOST}:{SOCKET_PORT}...")
    df = spark.readStream \
        .format("socket") \
        .option("host", SOCKET_HOST) \
        .option("port", SOCKET_PORT) \
        .load()

    # Parse JSON from the incoming data stream
    parsed_df = df.withColumn("data", from_json(col("value"), schema)).select("data.*")

    query = (
        parsed_df
        .writeStream
        .outputMode("append")
        .format("console")
        .start()
    )

    query.awaitTermination()

In [13]:
read_from_socket(spark)

Connecting to socket at localhost:9999...
-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+---------+----+---+-----+------+-------+
| id|title|timestamp|user|bot|minor|change|comment|
+---+-----+---------+----+---+-----+------+-------+
+---+-----+---------+----+---+-----+------+-------+

-------------------------------------------
Batch: 1
-------------------------------------------
+----------+-----------------+----------+--------------+-----+-----+------+--------------------+
|        id|            title| timestamp|          user|  bot|minor|change|             comment|
+----------+-----------------+----------+--------------+-----+-----+------+--------------------+
|1842145309|Talk:Legacy House|1731865478|OpalYosutebito|false|false|     9|Assessment: banne...|
+----------+-----------------+----------+--------------+-----+-----+------+--------------------+

-------------------------------------------
Batch: 2
----------------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/mpalamariuk/anaconda3/envs/mmds-project/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mpalamariuk/anaconda3/envs/mmds-project/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mpalamariuk/anaconda3/envs/mmds-project/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


-------------------------------------------
Batch: 10
-------------------------------------------
+----------+--------------------+----------+-----------------+-----+-----+------+--------------------+
|        id|               title| timestamp|             user|  bot|minor|change|             comment|
+----------+--------------------+----------+-----------------+-----+-----+------+--------------------+
|1842145328|Scottish Nosepickers|1731865484|UltrasonicMadness|false|false|   -17|Alternative spaci...|
+----------+--------------------+----------+-----------------+-----+-----+------+--------------------+



KeyboardInterrupt: 